# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
collection_list = db.list_collection_names()
print("Collections in uk_food:", collection_list)

Collections in uk_food: ['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [17]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
# Display the first document in the results using pprint
print(f"Number of establishments with a hygiene score of 20: {count}")
if count > 0:
    # Find and display the first document in the results using pprint
    result = establishments.find(query).limit(1)
    pprint(list(result)[0])
else:
    print("No establishments found with a hygiene score of 20.")

Number of establishments with a hygiene score of 20: 41
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': {'$toInt': '$RatingValue'},
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67326a5aadb464b3107e15f5'),
 'geocode': {'latitude': '50.769705', 'longitude': '0.27694'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/11068

In [18]:
# Execute the query and retrieve the results
results = establishments.find(query)

# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(results))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {df.shape[0]}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))


Number of rows in the DataFrame: 41
First 10 rows of the DataFrame:
                        _id   FHRSID  ChangesByServerID  \
0  67326a5aadb464b3107e15f5   110681                  0   
1  67326a5aadb464b3107e197a   612039                  0   
2  67326a5aadb464b3107e1c8a   730933                  0   
3  67326a5aadb464b3107e1e66   172735                  0   
4  67326a5aadb464b3107e1e80   172953                  0   
5  67326a5aadb464b3107e2825   512854                  0   
6  67326a5aadb464b3107e2a3c  1537089                  0   
7  67326a5badb464b3107e3f6b   155648                  0   
8  67326a5badb464b3107e43f3  1012883                  0   
9  67326a5badb464b3107e4bbf   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza 

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [20]:
# Find the establishments with London as the Local Authority and a RatingValue greater than or equal to 4
query = {"LocalAuthorityName": "London", "RatingValue": {"$gte": "4"}}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments in London with a RatingValue >= 4: {count}")

# Display the first document in the results using pprint
if count > 0:
    result = establishments.find(query).limit(1)
    pprint(list(result)[0])
else:
    print("No establishments found with the specified criteria.")


Number of establishments in London with a RatingValue >= 4: 0
No establishments found with the specified criteria.


In [21]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(results))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {df.shape[0]}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 0
First 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [25]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal "5"
# Sort by hygiene score in descending order

degree_search = 0.01
latitude = 51.148133  # Replace with your specific latitude
longitude = 1.383298  # Replace with your specific longitude

# Define the query to search establishments within the range of latitude and longitude and RatingValue = "5"
query = {
    "geocode.latitude": {"$gte": str(latitude - degree_search), "$lte": str(latitude + degree_search)},
    "geocode.longitude": {"$gte": str(longitude - degree_search), "$lte": str(longitude + degree_search)},
    "RatingValue": "5"
}

# Define the sorting by hygiene score in descending order
sort = [("scores.Hygiene", -1)]

# Set a limit for the results
limit = 10  # For example, limit to the top 10 results

# Find the results, sorted by hygiene score
results = establishments.find(query).sort(sort).limit(limit)

# Print the results
for result in results:
    pprint(result)


In [23]:
# Convert result to Pandas DataFrame
df = pd.DataFrame(list(results))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {df.shape[0]}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 0
First 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [3]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results

# Define the pipeline

pipeline = [
    # Step 1: Match establishments with a hygiene score of 0
    {"$match": {"hygiene_score": 0}},
    
    # Step 2: Group the matches by Local Authority and count the number of establishments in each
    {"$group": {
        "_id": "$local_authority",  # Group by Local Authority
        "count": {"$sum": 1}  # Count the number of establishments for each Local Authority
    }},
    
    # Step 3: Sort the matches by count in descending order (highest to lowest)
    {"$sort": {"count": -1}}
]

# Execute the aggregation pipeline
results = establishments.aggregate(pipeline)

# Convert the results to a list and count the number of documents
results_list = list(results)
print(f"Number of documents in the result: {len(results_list)}")

# Display the first 10 results
print("First 10 results:")
for result in results_list[:10]:
    pprint(result)


NameError: name 'establishments' is not defined

In [4]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(results))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {df.shape[0]}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

NameError: name 'pd' is not defined